# Word2Vec Tutorial

In case you missed the buzz, word2vec is a widely featured as a member of the “new wave” of machine learning algorithms based on neural networks, commonly referred to as "deep learning" (though word2vec itself is rather shallow). Using large amounts of unannotated plain text, word2vec learns relationships between words automatically. The output are vectors, one vector per word, with remarkable linear relationships that allow us to do things like vec(“king”) – vec(“man”) + vec(“woman”) =~ vec(“queen”), or vec(“Montreal Canadiens”) – vec(“Montreal”) + vec(“Toronto”) resembles the vector for “Toronto Maple Leafs”.

Word2vec is very useful in [automatic text tagging](https://github.com/RaRe-Technologies/movie-plots-by-genre), recommender systems and machine translation.

Check out an [online word2vec demo](http://radimrehurek.com/2014/02/word2vec-tutorial/#app) where you can try this vector algebra for yourself. That demo runs `word2vec` on the Google News dataset, of **about 100 billion words**.

## This tutorial

In this tutorial you will learn how to train and evaluate word2vec models on your business data.


## Preparing the Input
Starting from the beginning, gensim’s `word2vec` expects a sequence of sentences as its input. Each sentence is a list of words (utf8 strings):

In [1]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2017-05-07 14:19:35,470 : INFO : collecting all words and their counts
2017-05-07 14:19:35,473 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-07 14:19:35,474 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2017-05-07 14:19:35,476 : INFO : Loading a fresh vocabulary
2017-05-07 14:19:35,477 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-05-07 14:19:35,478 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-05-07 14:19:35,480 : INFO : deleting the raw counts dictionary of 3 items
2017-05-07 14:19:35,481 : INFO : sample=0.001 downsamples 3 most-common words
2017-05-07 14:19:35,483 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-05-07 14:19:35,484 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-05-07 14:19:35,485 : INFO : resetting layer weights
2017-05-07 14:19:35,487 : INFO : training model with 3 workers o

Keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large.

Gensim only requires that the input must provide sentences sequentially, when iterated over. No need to keep everything in RAM: we can provide one sentence, process it, forget it, load another sentence…

For example, if our input is strewn across several files on disk, with one sentence per line, then instead of loading everything into an in-memory list, we can process the input file by file, line by line:

In [3]:
# create some toy data to use with the following example
import smart_open, os

if not os.path.exists('./data/'):
    os.makedirs('./data/')

filenames = ['./data/f1.txt', './data/f2.txt']

for i, fname in enumerate(filenames):
    with smart_open.smart_open(fname, 'w') as fout:
        for line in sentences[i]:
            fout.write(line + '\n')

In [4]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

In [5]:
sentences = MySentences('./data/') # a memory-friendly iterator
print(list(sentences))

[['first'], ['sentence'], ['second'], ['sentence']]


In [6]:
# generate the Word2Vec model
model = gensim.models.Word2Vec(sentences, min_count=1)

2017-05-07 14:19:35,568 : INFO : collecting all words and their counts
2017-05-07 14:19:35,574 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-07 14:19:35,578 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2017-05-07 14:19:35,579 : INFO : Loading a fresh vocabulary
2017-05-07 14:19:35,582 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-05-07 14:19:35,587 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-05-07 14:19:35,588 : INFO : deleting the raw counts dictionary of 3 items
2017-05-07 14:19:35,589 : INFO : sample=0.001 downsamples 3 most-common words
2017-05-07 14:19:35,590 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-05-07 14:19:35,594 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-05-07 14:19:35,595 : INFO : resetting layer weights
2017-05-07 14:19:35,598 : INFO : training model with 3 workers o

In [7]:
print(model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'second': <gensim.models.keyedvectors.Vocab object at 0x106c55d50>, 'first': <gensim.models.keyedvectors.Vocab object at 0x106c55810>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x106c55b10>}


Say we want to further preprocess the words from the files — convert to unicode, lowercase, remove numbers, extract named entities… All of this can be done inside the `MySentences` iterator and `word2vec` doesn’t need to know. All that is required is that the input yields one sentence (list of utf8 words) after another.

**Note to advanced users:** calling `Word2Vec(sentences, iter=1)` will run **two** passes over the sentences iterator. In general it runs `iter+1` passes. By the way, the default value is `iter=5` to comply with Google's word2vec in C language. 
  1. The first pass collects words and their frequencies to build an internal dictionary tree structure. 
  2. The second pass trains the neural model.

These two passes can also be initiated manually, in case your input stream is non-repeatable (you can only afford one pass), and you’re able to initialize the vocabulary some other way:

In [8]:
# build the same model, making the 2 steps explicit
new_model = gensim.models.Word2Vec(min_count=1)  # an empty model, no training
new_model.build_vocab(sentences)                 # can be a non-repeatable, 1-pass generator     
new_model.train(sentences, total_examples=new_model.corpus_count, epochs=new_model.iter)                       
# can be a non-repeatable, 1-pass generator

2017-05-07 14:19:35,636 : INFO : collecting all words and their counts
2017-05-07 14:19:35,638 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-07 14:19:35,640 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2017-05-07 14:19:35,641 : INFO : Loading a fresh vocabulary
2017-05-07 14:19:35,644 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-05-07 14:19:35,645 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-05-07 14:19:35,646 : INFO : deleting the raw counts dictionary of 3 items
2017-05-07 14:19:35,647 : INFO : sample=0.001 downsamples 3 most-common words
2017-05-07 14:19:35,649 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-05-07 14:19:35,650 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-05-07 14:19:35,651 : INFO : resetting layer weights
2017-05-07 14:19:35,653 : INFO : training model with 3 workers o

0

In [9]:
print(new_model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'second': <gensim.models.keyedvectors.Vocab object at 0x106c55d50>, 'first': <gensim.models.keyedvectors.Vocab object at 0x106c55810>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x106c55b10>}


### More data would be nice
For the following examples, we'll use the [Lee Corpus](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee_background.cor) (which you already have if you've installed gensim):

In [10]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = test_data_dir + 'lee_background.cor'

In [11]:
class MyText(object):
    def __iter__(self):
        for line in open(lee_train_file):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

sentences = MyText()

print(sentences)

## Training
`Word2Vec` accepts several parameters that affect both training speed and quality.

### min_count
`min_count` is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them:

In [12]:
# default value of min_count=5
model = gensim.models.Word2Vec(sentences, min_count=10)

2017-05-07 14:19:35,718 : INFO : collecting all words and their counts
2017-05-07 14:19:35,721 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-07 14:19:35,765 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-05-07 14:19:35,766 : INFO : Loading a fresh vocabulary
2017-05-07 14:19:35,787 : INFO : min_count=10 retains 806 unique words (7% of original 10186, drops 9380)
2017-05-07 14:19:35,789 : INFO : min_count=10 leaves 40964 word corpus (68% of original 59890, drops 18926)
2017-05-07 14:19:35,795 : INFO : deleting the raw counts dictionary of 10186 items
2017-05-07 14:19:35,799 : INFO : sample=0.001 downsamples 54 most-common words
2017-05-07 14:19:35,802 : INFO : downsampling leaves estimated 26224 word corpus (64.0% of prior 40964)
2017-05-07 14:19:35,804 : INFO : estimated required memory for 806 words and 100 dimensions: 1047800 bytes
2017-05-07 14:19:35,812 : INFO : resetting layer weights
2017-05-07 14:1

### size
`size` is the number of dimensions (N) of the N-dimensional space that gensim Word2Vec maps the words onto.

Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

In [13]:
# default value of size=100
model = gensim.models.Word2Vec(sentences, size=200)

2017-05-07 14:19:36,122 : INFO : collecting all words and their counts
2017-05-07 14:19:36,125 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-07 14:19:36,159 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-05-07 14:19:36,161 : INFO : Loading a fresh vocabulary
2017-05-07 14:19:36,173 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2017-05-07 14:19:36,175 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2017-05-07 14:19:36,186 : INFO : deleting the raw counts dictionary of 10186 items
2017-05-07 14:19:36,188 : INFO : sample=0.001 downsamples 49 most-common words
2017-05-07 14:19:36,190 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2017-05-07 14:19:36,193 : INFO : estimated required memory for 1723 words and 200 dimensions: 3618300 bytes
2017-05-07 14:19:36,207 : INFO : resetting layer weights
2017-05-07 14:

### workers
`workers`, the last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) is for training parallelization, to speed up training:

In [14]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

2017-05-07 14:19:36,501 : INFO : collecting all words and their counts
2017-05-07 14:19:36,503 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-07 14:19:36,542 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-05-07 14:19:36,545 : INFO : Loading a fresh vocabulary
2017-05-07 14:19:36,561 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2017-05-07 14:19:36,564 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2017-05-07 14:19:36,574 : INFO : deleting the raw counts dictionary of 10186 items
2017-05-07 14:19:36,580 : INFO : sample=0.001 downsamples 49 most-common words
2017-05-07 14:19:36,582 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2017-05-07 14:19:36,583 : INFO : estimated required memory for 1723 words and 100 dimensions: 2239900 bytes
2017-05-07 14:19:36,598 : INFO : resetting layer weights
2017-05-07 14:

The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `word2vec` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).

## Memory
At its core, `word2vec` model parameters are stored as matrices (NumPy arrays). Each array is **#vocabulary** (controlled by min_count parameter) times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`Word2Vec` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. It is provided in the 'datasets' folder.

For example a syntactic analogy of comparative type is bad:worse;good:?. There are total of 9 types of syntactic comparisons in the dataset like plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as capital cities (Paris:France;Tokyo:?) or family members (brother:sister;dad:?). 

Gensim supports the same evaluation set, in exactly the same format:

In [15]:
model.accuracy('./datasets/questions-words.txt')

2017-05-07 14:19:36,892 : INFO : precomputing L2-norms of word weight vectors
2017-05-07 14:19:36,896 : INFO : family: 0.0% (0/2)
2017-05-07 14:19:36,924 : INFO : gram3-comparative: 0.0% (0/12)
2017-05-07 14:19:36,935 : INFO : gram4-superlative: 0.0% (0/12)
2017-05-07 14:19:36,949 : INFO : gram5-present-participle: 5.0% (1/20)
2017-05-07 14:19:36,967 : INFO : gram6-nationality-adjective: 0.0% (0/20)
2017-05-07 14:19:36,983 : INFO : gram7-past-tense: 0.0% (0/20)
2017-05-07 14:19:36,998 : INFO : gram8-plural: 0.0% (0/12)
2017-05-07 14:19:37,006 : INFO : total: 1.0% (1/98)


[{'correct': [], 'incorrect': [], 'section': u'capital-common-countries'},
 {'correct': [], 'incorrect': [], 'section': u'capital-world'},
 {'correct': [], 'incorrect': [], 'section': u'currency'},
 {'correct': [], 'incorrect': [], 'section': u'city-in-state'},
 {'correct': [],
  'incorrect': [(u'HE', u'SHE', u'HIS', u'HER'),
   (u'HIS', u'HER', u'HE', u'SHE')],
  'section': u'family'},
 {'correct': [], 'incorrect': [], 'section': u'gram1-adjective-to-adverb'},
 {'correct': [], 'incorrect': [], 'section': u'gram2-opposite'},
 {'correct': [],
  'incorrect': [(u'GOOD', u'BETTER', u'GREAT', u'GREATER'),
   (u'GOOD', u'BETTER', u'LONG', u'LONGER'),
   (u'GOOD', u'BETTER', u'LOW', u'LOWER'),
   (u'GREAT', u'GREATER', u'LONG', u'LONGER'),
   (u'GREAT', u'GREATER', u'LOW', u'LOWER'),
   (u'GREAT', u'GREATER', u'GOOD', u'BETTER'),
   (u'LONG', u'LONGER', u'LOW', u'LOWER'),
   (u'LONG', u'LONGER', u'GOOD', u'BETTER'),
   (u'LONG', u'LONGER', u'GREAT', u'GREATER'),
   (u'LOW', u'LOWER', u'GOOD',

This `accuracy` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.



In the December 2016 release of Gensim we added a better way to evaluate semantic similarity.

By default it uses an academic dataset WS-353 but one can create a dataset specific to your business based on it. It contains word pairs together with human-assigned similarity judgments. It measures the relatedness or co-occurrence of two words. For example, 'coast' and 'shore' are very similar as they appear in the same context. At the same time 'clothes' and 'closet' are less similar because they are related but not interchangeable.

In [16]:
model.evaluate_word_pairs(test_data_dir + 'wordsim353.tsv')

2017-05-07 14:19:37,057 : INFO : Pearson correlation coefficient against /usr/local/lib/python2.7/site-packages/gensim/test/test_data/wordsim353.tsv: 0.0819
2017-05-07 14:19:37,058 : INFO : Spearman rank-order correlation coefficient against /usr/local/lib/python2.7/site-packages/gensim/test/test_data/wordsim353.tsv: 0.0825
2017-05-07 14:19:37,060 : INFO : Pairs with unknown words ratio: 85.6%


((0.081883159986411394, 0.5678461885290379),
 SpearmanrResult(correlation=0.082498020328092989, pvalue=0.56493264964360379),
 85.55240793201133)

Once again, **good performance on Google's or WS-353 test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task. For an example of how to use word2vec in a classifier pipeline, see this [tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre).

## Storing and loading models
You can store/load models using the standard gensim methods:

In [17]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

2017-05-07 14:19:37,075 : INFO : saving Word2Vec object under /var/folders/4t/xx08nfg15lj77zlfjz69314r0000gn/T/tmpZEHE9Wgensim_temp, separately None
2017-05-07 14:19:37,078 : INFO : not storing attribute syn0norm
2017-05-07 14:19:37,079 : INFO : not storing attribute cum_table
2017-05-07 14:19:37,101 : INFO : saved /var/folders/4t/xx08nfg15lj77zlfjz69314r0000gn/T/tmpZEHE9Wgensim_temp


In [18]:
new_model = gensim.models.Word2Vec.load(temp_path)  # open the model

2017-05-07 14:19:37,107 : INFO : loading Word2Vec object from /var/folders/4t/xx08nfg15lj77zlfjz69314r0000gn/T/tmpZEHE9Wgensim_temp
2017-05-07 14:19:37,118 : INFO : loading wv recursively from /var/folders/4t/xx08nfg15lj77zlfjz69314r0000gn/T/tmpZEHE9Wgensim_temp.wv.* with mmap=None
2017-05-07 14:19:37,119 : INFO : setting ignored attribute syn0norm to None
2017-05-07 14:19:37,120 : INFO : setting ignored attribute cum_table to None
2017-05-07 14:19:37,121 : INFO : loaded /var/folders/4t/xx08nfg15lj77zlfjz69314r0000gn/T/tmpZEHE9Wgensim_temp


which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.

In addition, you can load models created by the original C tool, both using its text and binary formats:
```
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
  # using gzipped/bz2 input works too, no need to unzip:
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)
```

## Online training / Resuming training
Advanced users can load a model and continue training it with more sentences and [new vocabulary words](online_w2v_tutorial.ipynb):

In [19]:
model = gensim.models.Word2Vec.load(temp_path)
more_sentences = [['Advanced', 'users', 'can', 'load', 'a', 'model', 'and', 'continue', 'training', 'it', 'with', 'more', 'sentences']]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.iter)

# cleaning up temp
os.close(fs)
os.remove(temp_path)

2017-05-07 14:19:37,137 : INFO : loading Word2Vec object from /var/folders/4t/xx08nfg15lj77zlfjz69314r0000gn/T/tmpZEHE9Wgensim_temp
2017-05-07 14:19:37,146 : INFO : loading wv recursively from /var/folders/4t/xx08nfg15lj77zlfjz69314r0000gn/T/tmpZEHE9Wgensim_temp.wv.* with mmap=None
2017-05-07 14:19:37,147 : INFO : setting ignored attribute syn0norm to None
2017-05-07 14:19:37,149 : INFO : setting ignored attribute cum_table to None
2017-05-07 14:19:37,150 : INFO : loaded /var/folders/4t/xx08nfg15lj77zlfjz69314r0000gn/T/tmpZEHE9Wgensim_temp
2017-05-07 14:19:37,155 : INFO : collecting all words and their counts
2017-05-07 14:19:37,156 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-07 14:19:37,157 : INFO : collected 13 word types from a corpus of 13 raw words and 1 sentences
2017-05-07 14:19:37,158 : INFO : Updating model with new vocabulary
2017-05-07 14:19:37,159 : INFO : New added 0 unique words (0% of original 13)
                        and increas

You may need to tweak the `total_words` parameter to `train()`, depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C tool, `KeyedVectors.load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.

## Using the model
`Word2Vec` supports several word similarity tasks out of the box:

In [20]:
model.most_similar(positive=['human', 'crime'], negative=['party'], topn=1)

2017-05-07 14:19:37,190 : INFO : precomputing L2-norms of word weight vectors


[('longer', 0.9884582161903381)]

In [21]:
model.doesnt_match("input is lunch he sentence cat".split())

2017-05-07 14:19:37,202 : WARNING : vectors for words set(['lunch', 'input', 'cat']) are not present in the model, ignoring these words


'sentence'

In [22]:
print(model.similarity('human', 'party'))
print(model.similarity('tree', 'murder'))

0.999186470298
0.995724529077


You can get the probability distribution for the center word given the context words as input:

In [23]:
print(model.predict_output_word(['emergency', 'beacon', 'received']))

[('more', 0.001048518), ('continue', 0.00090946292), ('can', 0.00090134487), ('training', 0.00088478095), ('it', 0.00077986595), ('australia', 0.0007500046), ('there', 0.00074296352), ('government', 0.00074113585), ('could', 0.00073843176), ('or', 0.00073749834)]


The results here don't look good because the training corpus is very small. To get meaningful results one needs to train on 500k+ words.

If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [24]:
model['tree']  # raw NumPy vector of a word

array([ 0.00349002,  0.02440139, -0.02936695, -0.00849617, -0.03318483,
       -0.0382478 ,  0.00597728, -0.09292595, -0.01093712, -0.02097394,
        0.02088499, -0.0280605 , -0.07108893, -0.02044513, -0.02337479,
        0.04878484, -0.03198365, -0.00347298,  0.02429976, -0.02761379,
       -0.06878174, -0.00695439,  0.06986855,  0.05134906,  0.03044886,
        0.01195826, -0.00513146,  0.02122262, -0.01519287,  0.00502698,
        0.00088907,  0.07702309,  0.01296635, -0.00185401,  0.02448723,
        0.02151101, -0.04088883,  0.01947908,  0.01428026, -0.07242644,
       -0.08013999,  0.00214788, -0.04682875, -0.02618166,  0.03343621,
       -0.05884593,  0.03833489,  0.00581573, -0.01099163,  0.04513358,
        0.01407813,  0.00823141, -0.03918071,  0.0107606 ,  0.01743653,
       -0.01885621,  0.06017725, -0.03312737,  0.02473382, -0.03686444,
       -0.03306546, -0.05434534,  0.01816491, -0.0386038 , -0.01055549,
       -0.06602577, -0.08695736,  0.04147927,  0.05510609, -0.00

…or en-masse as a 2D NumPy matrix from `model.wv.syn0`.

## Conclusion

In this tutorial we learned how to train word2vec models on your custom data and also how to evaluate it. Hope that you too will find this popular tool useful in your Machine Learning tasks!

## Links


Full `word2vec` API docs [here](http://radimrehurek.com/gensim/models/word2vec.html); get [gensim](http://radimrehurek.com/gensim/) here. Original C toolkit and `word2vec` papers by Google [here](https://code.google.com/archive/p/word2vec/).